# 验证交易

使用Python实现交易验证过程：

In [1]:
# peer：节点，tx：待验证交易，pool：交易池
def verify_tx(peer,tx,pool = {}):
    
    # 验证交易基本信息
    if not verify_tx_basics(tx):
        return False

    # 验证交易池中是否存在双重支付
    if double_payment(pool,tx):
        logger.info('{0} double payment'.format(tx))
        return False
    
    # 初始化交易输入之和
    available_value = 0

    # 遍历交易所有的输入单元Vin
    for vin in tx.tx_in:
        
        # 在节点的UTXO_SET中查找该交易输入单元使用的UTXO
        utxo = peer.utxo_set.get(vin.to_spend)
        
        # 如果UTXO不存在，将该交易添加到孤立交易池中
        if not utxo:
            logger.info(
                    '{0}(pid={1}) find a orphan transaction {2}'.format(peer,peer.pid,tx)
                    )
            peer.orphan_pool[tx.id] = tx
            return False
        
        # 对每个输入单元验证签名，如果不成功则拒绝交易
        if not verify_signature(peer,vin,utxo,tx.tx_out):
            logger.info('singature does not math for {0}'.format(tx))
            return False
        
        # 计算交易输入金额总和
        available_value += utxo.vout.value
        
    # 如果交易输入金额总和小于交易输出金额总和，则拒绝交易
    if available_value < sum(vout.value for vout in tx.tx_out):
        logger.info(
                '{0} no enought available value to spend by {1}(pid={2})'.format(tx,peer,peer.pid)
                )
        return False

    # 所有条件满足则验证交易
    return True

验证交易基本条件过程：

In [2]:
# tx：待验证交易
def verify_tx_basics(tx):
    
    # 验证交易类型Tx
    if not isinstance(tx,Tx):
        logger.info('{0} is not Tx type'.format(tx))
        return False
    
    # 交易输入输出的非空验证
    if (not tx.tx_out) or (not tx.tx_in):
        logger.info('{0} Missing tx_out or tx_in'.format(tx))
        return False
    return True

验证交易池中是否存在双重支付，即判断待验证的交易与交易池中的交易是否使用了相同的UTXO。

In [3]:
# pool：节点的交易池，tx：待验证交易
def double_payment(pool,tx):
    
    # 验证交易是否已经在交易池中
    if tx.id in pool:
        return True
    
    # 将待验证交易输入单元的定位指针存储在集合a中
    a = {vin.to_spend for vin in tx.tx_in}
    
    # 将交易池中所有交易输入单元的定位指针存储在集合b中
    b = {vin.to_spend for tx in pool.values() for vin in tx.tx_in}
    
    # 如果集合a和b有交集，则为双重支付
    return a.intersection(b)

### 示例

In [4]:
from simchain import Network
from simchain.peer import verify_tx_basics, double_payment, verify_tx

net = Network()
zhangsan, lisi = net.peers[0], net.peers[1]

# 张三给李四转账100
zhangsan.create_transaction(lisi.addr, 100)
# 获取张三创建的交易
tx = zhangsan.txs[-1]
# 验证交易基本条件满足
verify_tx_basics(tx)

2019-06-11 23:49:44,100 - A blockchain p2p network created,12 peers joined
2019-06-11 23:49:44,131 - genesis block has been generated
2019-06-11 23:49:44,176 - peer(52, 42)(pid=0) created a transaction


True

In [5]:
# 没有双重支付发生
double_payment(zhangsan.mem_pool, tx)

set()

此时张三并没有广播交易，如果广播交易后会发生什么呢？

In [6]:
# 张三广播交易
zhangsan.broadcast_transaction()
# 张三的交易池
zhangsan.mem_pool

2019-06-11 23:49:44,317 - peer(52, 42)(pid=0) sent a transaction to network
2019-06-11 23:49:45,180 - peer(52, 42)(pid=0)'s transaction verified by 9 peers


{'9f0af61720abeaf459a801b1966e67760110fa0e03310687a0eb4a10b4ddc8ed': Tx(id:9f0af61720abeaf459a801b1966e67760110fa0e03310687a0eb4a10b4ddc8ed)}

In [7]:
# 李四和张三一样的交易池
lisi.mem_pool

{}

In [8]:
# 张三验证交易不通过
verify_tx(zhangsan, tx, zhangsan.mem_pool)

2019-06-11 23:49:45,299 - Tx(id:9f0af61720abeaf459a801b1966e67760110fa0e03310687a0eb4a10b4ddc8ed) double payment


False

In [9]:
# 因为该条交易已经存在交易池中
double_payment(zhangsan.mem_pool, tx)

True

## 数字签名匹配

使用Python实现单个输入单元的验证过程：

In [10]:
# vin：待验证交易的输入单元，utxo：需要消费的UTXO，tx_out：交易的输出
def verify_signature_for_vin(vin,utxo,tx_out):
    
    # 获取输入单元中的签名，以及公钥字节串
    pk_str,signature = vin.pubkey,vin.signature
    
    # 获取UTXO指向的地址
    to_addr = utxo.vout.to_addr
    
    # 用UTXO的定位指针、公钥字节符，以及交易输出作为签名明文
    # 选用的明文必须与创建交易时的规则一致
    # 调用build_message()函数计算明文的哈希值
    string = str(vin.to_spend) + str(pk_str) + str(tx_out)
    message = build_message(string)
    
    # 将公钥字节符转换为地址
    pubkey_as_addr = convert_pubkey_to_addr(pk_str)
    
    # 从公钥字节符创建公钥对象
    verifying_key = VerifyingKey.from_bytes(pk_str)

    # 如果通过公钥字节串转换得到的地址与UTXO指向的地址不匹配，则拒绝
    # 简单来讲，UTXO中指向的地址与公钥不是对应关系
    if pubkey_as_addr != to_addr:
        return False
    
    # 如果通过公钥字节串创建的公钥对象验证签名不通过，则拒绝
    # 简单来讲，公钥和私钥不是对应关系
    if not verifying_key.verify(signature, message):
        return False

    # 所有条件满足，则通过签名验证
    return True

### 继续示例代码

In [11]:
from simchain.peer import verify_signature_for_vin

# 获取交易的第一个输入单元
vin = tx.tx_in[0]
# 获取输入单元使用的UTXO
utxo = zhangsan.utxo_set[vin.to_spend]
# 验证输入单元有效
verify_signature_for_vin(vin, utxo, tx.tx_out)

True

当交易被独立验证有效后，节点会将其加入交易池中。除此之外，还要对孤立交易池进行验证。

## 创币交易的验证

In [12]:
# tx：交易，reward：总奖励（共识奖励和交易费之和）
def verify_coinbase(tx,reward):
    
    # 是否为Tx类型
    if not isinstance(tx,Tx):
        return False
    
    # 是否为创币交易
    if not tx.is_coinbase:
        return False
    
    # 交易的输出列表是否只有一个输出单元，输出金额是否等于总金额
    if (not (len(tx.tx_out) ==1))  or (tx.tx_out[0].value != reward):
        return False
    return True

----
# 交易传播中的安全

## 签名明文攻击

假设张三离线创建了一条指向李四的交易，然后将其广播到了区块链网络中，张三的邻居节点只有王五，王五截获并将交易输出替换成自己的地址，然后将修改后的新交易传播到网络中。示例如下：

In [13]:
from simchain import Network, Vout
net = Network(nop=2, von=10000)
zhangsan, lisi = net.peers[0], net.peers[1]
zhangsan.create_transaction(lisi.addr, 1000)
net.add_peer()
wangwu = net.peers[2]
# 获取张三创建的交易
tx = zhangsan.txs[-1]
tx

2019-06-11 23:49:45,962 - A blockchain p2p network created,2 peers joined
2019-06-11 23:49:45,962 - genesis block has been generated
2019-06-11 23:49:46,012 - peer(45, 70)(pid=0) created a transaction
2019-06-11 23:49:46,054 - A new peer joined in --> peer(33, 1)(pid=2)


Tx(id:3b6a3bd60071eafa0c9f59c431d47c4adcc5dc5c7e0ae0b5f86c1e4ad0120e1a)

In [14]:
# 获取交易的第一个输出单元
vout = tx.tx_out[0]
# 王五替换地址
new_vout = Vout(wangwu.addr, 1000)
# 替换交易的第一个输出单元，得到新的交易
tx.tx_out[0] = new_vout
# 交易编号发生变化
tx

Tx(id:cf87b08e465fe46bcfc27bf365e35706a7d05db7a9f99df685b0c407ee910759)

In [15]:
# 王五将交易发送给李四，李四验证不通过
lisi.verify_transaction(tx)

2019-06-11 23:49:46,236 - singature does not math for Tx(id:cf87b08e465fe46bcfc27bf365e35706a7d05db7a9f99df685b0c407ee910759)


False

如果某种加密货币的客户端在数字签名的过程中选择的签名明文如果与交易的输出无关，例如仅选择输入单元的定位指针、公钥，而不包括交易的输出，则表明网络中所有有效交易的输出都可以被篡改的，这种攻击称为签名明文攻击。

### 示例

In [16]:
from simchain import Pointer, Vin, Vout, Tx
from simchain.ecc import SigningKey, convert_pubkey_to_addr, VerifyingKey

# 假设这是张三的私钥
k = 345643
# 用整数创建一个私钥对象
sk = SigningKey.from_number(k)
# 获取该私钥对应的公钥
pk = sk.get_verifying_key()
# 将公钥转换为对应的地址
addr = convert_pubkey_to_addr(pk.to_bytes())
# 用创币交易给张三发100的钱
coinbase = Tx.create_coinbase(addr, 100)
# 获取张三创币交易中的UTXO的定位指针
pointer = Pointer(coinbase.id, 0)
# 张三选择一条签名明文
message = b'I love blockchain'
# 张三用该明文签名
sig = sk.sign(message)
# 张三创建了一个输入单元
vin = Vin(pointer, sig, pk.to_bytes())
# 李四对该输入单元进行验证，先获取公钥
pub_str = vin.pubkey
# 将公钥转换为地址
addr1 = convert_pubkey_to_addr(pub_str)
# 地址匹配
addr1 == addr

True

In [17]:
# 李四用公钥字节串创建一个公钥对象
vk = VerifyingKey.from_bytes(pub_str)
# 选择与张三一样的签名明文验证签名通过
vk.verify(sig, message)

True

In [18]:
message1 = b'i love blockchain'
# 选择与张三不一样的明文验证签名不通过
vk.verify(sig, message1)

False